In [1]:
import numpy as np

In [2]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))


def relu(x):
    return np.maximum(0, x)


def softmax(x):
    if x.ndim == 2:
        x = x - x.max(axis=1, keepdims=True)
        x = np.exp(x)
        x /= x.sum(axis=1, keepdims=True)
    elif x.ndim == 1:
        x = x - np.max(x)
        x = np.exp(x) / np.sum(np.exp(x))

    return x


def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
        
    # 在监督标签为one-hot-vector的情况下，转换为正确解标签的索引
    if t.size == y.size:
        t = t.argmax(axis=1)
             
    batch_size = y.shape[0]

    return -np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size

In [3]:
class Sigmoid:
    def __init__(self):
        self.params = []
        self.grads = []
        self.out = None

    def forward(self,x):
        out = sigmoid(x)
        self.out = out
        return out
    def backward(self,dout):
         dx = dout * (1-self.out) * self.out
         return dx
         

    

In [4]:
class Affine:
    def __init__(self,W,b):
        self.params = [W,b]
        self.grads = [np.zeros_like(W),np.zeros_like(b)]
        self.matmul_layer = MatMul(W)
        self.x = None

    def forward(self,x):
        W,b = self.params
        out = self.matmul_layer.forward(x) + b        
        self.x = x
        return out        

    def backward(self,dout):
        W,b = self.params
        
        dx = self.matmul_layer.backward(dout)
        dW = self.matmul_layer.grads[0]
        db = np.sum(dout,axis=0)
        
        self.grads[0][...] = dW
        self.grads[1][...] = db
        return dx

In [5]:
class TwoLayerNet:
    def __init__(self, input_size,hidden_size, output_size):
        I, H, O = input_size, hidden_size, output_size


        W1 = 0.01 * np.random.randn(I,H)
        b1 = np.zeros(H)
        W2 = 0.01 * np.random.randn(H,O)
        b2 = np.zeros(O)

        self.layers = [Affine(W1,b1),Sigmoid(),Affine(W2,b2)]

        self.loss_layer = SoftmaxWithLoss()

        self.params, self.grads = [] , []

        for layer in self.layers:
            self.params += layer.params
            self.grads += layer.grads
    
    def forward(self,x,t):
        score = self.predict(x)
        loss = self.loss_layer.forward(score,t)
        return loss
    
    
    
    def predict(self,x):
        for layer in self.layers:
            x = layer.forward(x)
        return x 

    def backward(self,dout=1):
        dout = self.loss_layer.backward(dout)
        for layer in reversed(self.layers):
            dout = layer.backward(dout)
        return dout
        

In [6]:
class SoftmaxWithLoss:
    def __init__(self):
        self.params, self.grads = [], []
        self.y = None  # softmax的输出
        self.t = None  # 监督标签

    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)

        # 在监督标签为one-hot向量的情况下，转换为正确解标签的索引
        if self.t.size == self.y.size:
            self.t = self.t.argmax(axis=1)

        loss = cross_entropy_error(self.y, self.t)
        return loss

    def backward(self, dout=1):
        batch_size = self.t.shape[0]

        dx = self.y.copy()
        dx[np.arange(batch_size), self.t] -= 1
        dx *= dout
        dx = dx / batch_size

        return dx
        

In [7]:
class SGD:
    def __init__(self,lr=0.01):
        self.lr = lr
    def update(self,params,grads):
        for i in range(len(params)):
            params[i] -= self.lr * grads[i]
        

In [8]:
class MatMul:
    def __init__(self,W):
        self.params = [W]
        self.grads = [np.zeros_like(W)]
        self.x = None
    def forward(self,x):
        W ,= self.params
        out = np.dot(x,W)
        self.x = x
        return out
    
    def backward(self,dout):
        W, = self.params
        dx = np.dot(dout,W.T)
        dW = np.dot(self.x.T,dout)
        self.grads[0][...] = dW
        return dx         

repeate 节点

N, D = 8, 7

forward

x = np.random.randn(1,D)
y = np.repeat(x,N,axis=0)

backward

dy = np.random.randn(N,D)
dx = np.sum(dy,axis=0,keepdims=True)

sum 节点
D, N = 8, 7

forward
x = np.random.randn(N,D)
y = np.sum(x,axis=0,keepdims=True)

backward
dy = np.random.randn(1,D)
dx = np.repeat(dy,N,axis=0)

In [9]:
def load_data(seed=1984):
    np.random.seed(seed)
    N = 100  # 各类的样本数
    DIM = 2  # 数据的元素个数
    CLS_NUM = 3  # 类别数

    x = np.zeros((N*CLS_NUM, DIM))
    t = np.zeros((N*CLS_NUM, CLS_NUM), dtype=np.int)

    for j in range(CLS_NUM):
        for i in range(N):#N*j, N*(j+1)):
            rate = i / N
            radius = 1.0*rate
            theta = j*4.0 + 4.0*rate + np.random.randn()*0.2

            ix = N*j + i
            x[ix] = np.array([radius*np.sin(theta),
                              radius*np.cos(theta)]).flatten()
            t[ix, j] = 1

    return x, t


In [10]:
import matplotlib.pyplot as plt

if __name__ == '__main__':
    max_epoch = 300
    batch_size = 30
    hidden_size = 10
    learning_rate = 1.0

    x, t = load_data()
    model = TwoLayerNet(2,hidden_size,3)
    optimizer = SGD(lr=learning_rate)

    data_size = len(x)

    max_iters = data_size // batch_size
    total_loss = 0
    loss_count = 0
    loss_list = []

    for epoch in range(max_epoch):
        idx = np.random.permutation(data_size)
        x = x[idx]
        t = t[idx]
        for iters in range(max_iters):
            batch_x = x[iters*batch_size:(iters+1)*batch_size]
            batch_t = t[iters*batch_size:(iters+1)*batch_size]
            loss = model.forward(batch_x,batch_t)
            model.backward()
            optimizer.update(model.params,model.grads)
            total_loss += loss
            loss_count += 1

            if (iters + 1) % 10 ==0:
                avg_loss = total_loss / loss_count
                print("| epoch %d | iter %d / %d | loss %.2f" % 
                        (epoch + 1, iters + 1, max_iters, avg_loss))
                loss_list.append(avg_loss)
                total_loss, loss_count = 0,0
    

<ipython-input-9-4d18fe3988f6>:8: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  t = np.zeros((N*CLS_NUM, CLS_NUM), dtype=np.int)


ValueError: shapes (2,300) and (30,3) not aligned: 300 (dim 1) != 30 (dim 0)